In [43]:
from groq import Groq
import base64


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "./save_dir/Switz/IMG_0280/frames/frame_0001.png"

# Getting the base64 string
base64_image = encode_image(image_path)

client = Groq()

metadata= {
    "duration": 4,
    "created": "2024-09-19T08:03:09Z",
    "location": "21, Limmatstrasse, Gewerbeschule, Industriequartier, Zurich, District Zurich, Zurich, 8005, Switzerland",
    "framerate": 30.0
}

def get_system_prompt(location:str)->str:
  return f"""
You are a helpful AI assistant, who is an expert in describing the image. 
Carefully describe the given image. 
Additionally, incorporate the location information ({location}) to provide relevant contextual details.
"""

def get_image_description(sysem_prompt: str, base64_image: bytes) -> str:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    # {"type": "text", "text": f"Describe the given image. This image is taken at {metadata['location']}. Use the location information also in the description"},
                    {"type": "text", "text": sysem_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        model="llama-3.2-11b-vision-preview",
    )

    return chat_completion.choices[0].message.content

system_prompt = get_system_prompt(location=metadata["location"])
get_image_description(system_prompt, base64_image)

'The image depicts a blue tram traveling down a street in Zurich, Switzerland. \n\n* The tram is blue and white with a glass front and a flat roof, featuring a sign that reads "Frankental" at the front. \n* It is positioned on the left side of the image, facing forward, and is situated on a road with two lanes. \n* The pavement is divided into two sections by a dotted line, with a large, light gray section on the right and a dark gray section on the left. \n* The street is lined with buildings on both sides, predominantly dark-colored with multiple windows. \n* Above the tram, there is a large orange construction crane extending from the right side of the image to the left. \n* The background shows an overcast sky, with a gradient of light blue at the top. \n* At the very top of the image, there is a black wire with a single light bulb hanging from it, flanked by two small lights. \n* The overall atmosphere suggests a daytime setting with an overcast sky.'

In [ ]:
from pathlib import Path
import json

file_directory = Path("./save_dir/Switz/IMG_0280")
frames_directory = file_directory / "frames"
frame_description_save_directory = file_directory / "descriptions"
frame_description_save_directory.mkdir(exist_ok=True)

with open(file_directory / "metadata.json", "r") as f:
    metadata = json.load(f)

for image_path in frames_directory.iterdir():
    base64_image = encode_image(image_path)
    location = metadata['location']
    system_prompt = get_system_prompt(location)
    frame_description = get_image_description(system_prompt, base64_image)
    frame_description_file_path = frame_description_save_directory / f"{image_path.stem}.txt"
    with open(frame_description_file_path, "w") as f:
        f.write(frame_description)
    print(f"Frame description written to {frame_description_file_path}")


frame_0001
Frame description written to save_dir/Switz/IMG_0280/descriptions/frame_0001.txt
frame_0003
Frame description written to save_dir/Switz/IMG_0280/descriptions/frame_0003.txt
frame_0002
Frame description written to save_dir/Switz/IMG_0280/descriptions/frame_0002.txt
frame_0006
Frame description written to save_dir/Switz/IMG_0280/descriptions/frame_0006.txt
frame_0005
Frame description written to save_dir/Switz/IMG_0280/descriptions/frame_0005.txt
frame_0004
Frame description written to save_dir/Switz/IMG_0280/descriptions/frame_0004.txt
